# Malware triage with MISP - Dynamic malware analysis

## Introduction

- UUID: **8bada769-763a-46c3-a9b2-6b4808a47d36**
- Started from [issue 3](https://github.com/MISP/misp-playbooks/issues/3)
- State: **Published** : demo version with **output**
- Purpose: This playbook extends the results retrieved with **static** malware analysis in the [malware triage](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage.ipynb) playbook and does the **dynamic** malware analysis with one or more **malware sandboxes**. 
    - The results are stored in a **MISP report** and sent to Mattermost.
- Tags: [ "malware", "triage", "sandbox", "incidentresponse", "ir", "dfir" ]
- External resources: **VMRay**, **Hybrid-Analysis**, **VirusTotal**, **Mattermost**
- Target audience: **SOC**, **CSIRT**, **CTI**
- Notes:
    - First run the [malware triage](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage.ipynb) playbook for **static** analysis,
    - Then run this playbook for **dynamic** analysis

# Playbook

- **Malware triage with MISP - Dynamic malware analysis**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Verify MISP modules
    - PR:3 Load helper functions
    - PR:4 Set helper variables
    - PR:5 Setup MISP event link
    - PR:6 Load the file objects
- **Investigate**
    - IN:1 Submit to VMRay
    - IN:2 Submit to Hybrid-Analysis
    - IN:3 Submit to VirusTotal
    - IN:4 Create MISP report for investigation
- **Summary**
    - EN:1 MISP indicators
    - EN:2 Print the dynamic malware analysis report
    - EN:3 Create the summary of the playbook
    - EN:4 Send a summary to Mattermost
    - EN:5 End of the playbook
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Ignore certificate errors
vmray_apikey=<KEY>                     # VMRay API key
hybridanalysis_apikey=<KEY>            # Hybrid-analysis key
virustotal_apikey=<KEY>                # VirusTotal key
```

In [25]:
# Initialise Python environment
import urllib3
import sys
import json
from pyfaup.faup import Faup
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
import uuid
#from uuid import uuid4
from pymisp import *
from pymisp.tools import GenericObjectGenerator
from pymisp.tools import make_binary_objects

import os
import time
import pefile
from datetime import datetime

import io
import zipfile
import base64

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

# Create headers for specific cases
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}

The version of PyMISP recommended by the MISP instance (2.4.176) is newer than the one you're using now (2.4.173). Please upgrade PyMISP.


The Python libraries are loaded and the credentials are read from the keys file.
I will use the MISP server https://misp.demo.cudeso.be/ for this playbook.




## PR:2 Verify MISP modules

This playbook uses the MISP modules to obtain additional correlation or enrichment information. [MISP modules](https://github.com/MISP/misp-modules) are autonomous modules that can be used to extend MISP for new services such as expansion, import and export. The modules are written in Python 3 following a simple API interface. The objective is to ease the extensions of MISP functionalities without modifying core components. The API is available via a simple REST API which is independent from MISP installation or configuration.

In the next cell we check if we have access to the **MISP module** server and if the required modules are enabled.

In [2]:
# Where can we find the local MISP Module server? You can leave this to the default setting in most cases.
misp_modules_url = "http://127.0.0.1:6666"

# How long do we wait between queries when using the MISP modules (API rate limiting of external service such as VirusTotal)
misp_modules_wait = 3

# Initiliasation
misp_modules = {}
misp_modules_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
misp_modules_in_use = ["vmray_import"]
# Code block to query the MISP module server and check if our modules are enabled
res = requests.get("{}/modules".format(misp_modules_url), headers=misp_modules_headers)
for module in res.json():
    for module_requested in misp_modules_in_use:
        if module.get("name", False) == module_requested:
            misp_modules[module_requested] = {"enabled": True, "input": module.get("mispattributes").get("input")}
            print("Found the \033[92m{}\033[90m MISP module (Accepted input: {}).".format(module_requested, misp_modules[module_requested]["input"]))
print("\n")

Found the vmray_import MISP module (Accepted input: None).




## PR:3 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

Instead of distributing helper functions as separate Python files this playbook includes all the required code as one code cell. This makes portability of playbooks between instances easier. The downside is that functions defined in this playbook need to be defined again in other playbooks, which is not optimal for code re-use. For this iteration of playbooks it is chosen to include the code in the playbook (more portability), but you can easily create one "helper" file that contains all the helper code and then import that file in each playbook (for example by adding to the previous cell `from helpers import *`). Note that the graphical workflow image is included as an external image. A missing image would not influence the further progress of the playbook.

In [3]:
def sandbox_calculate_risk_score(ratio):
    '''
    Calculate risk score and return the tag
    '''
    if ratio > 0:
        if ratio >= 75:
            tag = "misp:threat-level=\"high-risk\""
        elif ratio >= 50:
            tag = "misp:threat-level=\"medium-risk\""
        #elif detection_ratio_perc >= 25:
        else:
            tag = "misp:threat-level=\"low-risk\""
    else:
        tag = "misp:threat-level=\"no-risk\""
    return tag

## PR:4 Set helper variables

This cell contains **helper variables** that are used in this playbook. Their usage is explained in the next steps of the playbook.

- `samples` : a dictionary of the objects that are created when the playbook progresses
- `submission_tags` : tags that are used when uploading samples to the sandbox

In [4]:
# Dictionary to store playbook results
samples = {}

# Tags to add to the malware submission
submission_tags = ["MISP", "playbook", "malwaretriage"]

# Helper variables
misp_event = False
summary = ""

## PR:5 Setup MISP event link

This playbook does **not** create a new MISP event. It relies on an existing event and loads the file objects from that event. Ideally you start your investigation with a **static** analysis of the sample through the MISP playbook [Malware Triage](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage.ipynb). Then, if the static analysis did not provide sufficient information, you can use malware sandboxes for **dynamic** analysis.

In the next cell, provide the MISP event **ID** or **UUID** in the variable `event_link`.

In [5]:
# Provide an event ID or an event UUID
event_link = "97c61f93-a8e7-43a2-807a-1a7fe170f4db"

# Get the MISP event
misp_event = misp.get_event(event_link, pythonify=True)
if not "errors" in misp_event:
    print("Fetched event \033[92m{}\033[90m with title \033[92m{}\033[90m.".format(event_link, misp_event.info))
    print("Continue with the playbook.\n")
else:
    print("Unable to get event with identifier \033[91m{}\033[90m".format(event_link))
    print("It's not possibel to continue with the playbook.\n")

Fetched event 97c61f93-a8e7-43a2-807a-1a7fe170f4db with title Malware triage for incident in L2/L3.
Continue with the playbook.



## PR:6 Load file objects

Instead of loading samples from disk, the playbook loads the samples via the MISP **file objects**, from the event you identified earlier in `event_link` (and now stored in `misp_event`).

In [6]:
print("Searching for files in event {} {}".format(misp_event.info, misp_event.id))
for event_object in misp_event.Object:
    if event_object.name == "file":
        print("Found file object ...")
        for attr in event_object.Attribute:
            if attr.object_relation == "malware-sample":
                print(" As malware-sample ...")
                malware_sample = True
                key = attr.value.split("|")[0]
                print("  File \033[92m{}\033[90m with UUID {}".format(key, attr.uuid))
                samples[key] = {}
                samples[key]["fileobject"] = event_object
                samples[key]["fileobject_uuid"] = event_object.uuid

                for attribute in event_object.Attribute:
                    if attribute.object_relation == "md5":
                        samples[key]["md5"] = attribute.value
                        print("  MD5: \033[92m{}\033[90m".format(samples[key]["md5"]))
                    if attribute.object_relation == "sha256":
                        samples[key]["sha256"] = attribute.value
                        print("  SHA256: \033[92m{}\033[90m".format(samples[key]["sha256"]))
                samples[key]["data"] = attr.data
                # Prepare dictionary for the sandboxes
                samples[key]["hybrid-analysis"] = {}
                samples[key]["vmray"] = {}
                samples[key]["virustotal"] = {}

print("\n")
for key, sample in samples.items():
    print("Obtaining data for \033[92m{}\033[90m".format(key))
    zf = zipfile.ZipFile(sample["data"])
    sample_hashname = zf.namelist()[0]
    data = zf.read(sample_hashname, b"infected")
    zf.close()
    sample["submitdata"] = data
    print(" Got {} bytes".format(len(data)))

print("\n")
for key, sample in samples.items():
    if len(sample["submitdata"]) > 0:
        print("File \033[92m{}\033[90m is ready to be submitted to sandboxes".format(key))

Searching for files in event Malware triage for incident in L2/L3 3138
Found file object ...
 As malware-sample ...
  File 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run with UUID 7d7339c5-e616-4b05-87fa-aa2e2654a966
  MD5: 7c9da2a2c29c0411e614322a02bc43fa
  SHA256: 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4


Obtaining data for 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
 Got 115256 bytes


File 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run is ready to be submitted to sandboxes


# Investigate

This section starts the investigation by submitting the samples to sandboxes and consists of two steps:

- First **submit** the sample to the sandbox so that the analysis jobs can start;
- And afterwards, once the analysis jobs have finished, **collect the results**.

Depending on your preference you can first execute all the submit sections and later, when the jobs have finished, come back and get collect the results.

Before submitting a sample to a sandbox make sure that you are allowed to share it outside your organisation. When in doubt, use [PAP - Permissible Actions Protocol](https://www.circl.lu/doc/misp-taxonomies/#_pap) to describe where you can use the information. Also see the post on [How to Support Defenders with the Permissible Actions Protocol](https://www.vanimpe.eu/2020/12/28/how-to-support-defenders-with-the-permissible-actions-protocol/).

## IN:1 Submit to VMRay

The first sandbox is [VMRay](https://www.vmray.com/). There are a number of configuration settings for the submission:
- Share the hash with **VirusTotal** (set `vmray_shareable` to True)
- Use **reputation analysis** (`vmray_enable_reputation`)
- You can **ignore specific objects** from being imported in the MISP event (with `vmray_ignore_objects`). This is for example useful to ignore the list of processes if you do not have detection capabilities to track process creation
- Include the sandbox **signatures** (with `vmray_signatures_in_report`). In VMRay, the signatures are part of the sb-sandbox object and can be quite comprehensive. For quick and easy-to-read reports it's advised to ignore them.

The number and type of jobs executed for analysis depend on your VMRay configuration. Submitted files create new jobs according to the default **Jobrules** of the sample type. The parameter `vmray_jobrule_entries` can be used to specify alternative Jobrules. If you leave the variable to False, your default VMRay configuration is used.

In [7]:
# Share the hash with VirusTotal
#vmray_shareable = True
vmray_shareable = False

# Do reputation analysis
vmray_enable_reputation = True

# ignore these objects when importing the results
vmray_ignore_objects = ["process"]

# Include signatures in report?
vmray_signatures_in_report = False

# VMRay headers
vmray_headers = {
    "accept": "application/json",
    "Authorization": "api_key {}".format(vmray_apikey)
}

# Job rules
# Example: ["vmray:win10:def:adobe_reader-8", "vmray:win10:def:adobe_reader-6"]
#          {"Windows PE (x86-32)": ["vmray:win10_64_th2:def:exe", "vmray:win7_32_sp1:def:exe"], "PDF Document": ["vmray:win10_64_th2-pdf:def:adobe-reader-64-DC-2018.011.20038"]}
vmray_jobrule_entries = False

### Submit

Execute the next cell to sumit the samples to VMRay. This will output
- VMRay **jobs**
- **Submission** ID
- **Sample** ID

In [8]:
# Code block to submit sample to VMRay
for key, sample in samples.items():
    print("Submit \033[92m{}\033[90m to VMRay.".format(key))

    tags = ""
    for tag in submission_tags:
        tags = "{}{},".format(tags, tag)

    submit_parameters = {
        "tags": tags[:-1],
        "shareable": vmray_shareable,
        "enable_reputation": vmray_enable_reputation
        }
    if vmray_jobrule_entries:
        submit_parameters["jobrule_entries"] = vmray_jobrule_entries
    submit_file = {"sample_file": (key, io.BytesIO(sample["submitdata"]))}
    response = requests.post("{}/rest/sample/submit".format(vmray_url), headers=vmray_headers, files=submit_file, data=submit_parameters)

    if response.status_code == 200:
        if response.json().get("result") == "ok":
            response_json = response.json().get("data", {})
            jobs = response_json.get("jobs", [])
            if len(jobs) > 0:
                print(" VMRay jobs")
                for job in jobs:
                    print("  {} \033[92m{}\033[90m {}".format(job["job_type"], job["job_id"], job["job_vm_name"]))

            submissions = response_json.get("submissions", [])
            if len(submissions) > 0:
                print(" VMRay submissions")
                samples[key]["vmray"]["submissions"] = []
                for submission in submissions:
                    print("  \033[92m{}\033[90m".format(submission["submission_id"]))
                    samples[key]["vmray"]["submissions"].append(submission["submission_id"])

            vmray_samples = response_json.get("samples", [])
            if len(vmray_samples) > 0:
                print(" VMRay samples")
                samples[key]["vmray"]["samples"] = []
                for vmray_sample in vmray_samples:
                    print("  \033[92m{}\033[90m".format(vmray_sample["sample_id"]))
                    samples[key]["vmray"]["samples"].append(vmray_sample["sample_id"])
            print("Successfully submitted malware sample.")
        else:
            print("Failed to submit malware sample. Status Code: \033[91m{}\033[90m {}".format(response.status_code, response.json().get("result")))
            print(response.text)

Submit 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run to VMRay.
 VMRay submissions
  13873809
 VMRay samples
  12102735
Successfully submitted malware sample.


### Collect results

Analysing a sample in a sandbox takes time. Execute the next cell to collect the results. The playbook checks if all jobs for your sample are finished. Once they are finished, it will get the results and use the **MISP VMRay import module** to import the sandbox results into the MISP event.

In [13]:
# Collect the VMRay results
for key, sample in samples.items():
    print("Process VMRay \033[92m{}\033[90m".format(key))
    report_md = ""

    # First check the submission is finished
    submission_finished = False
    for vmray_submission in sample["vmray"]["submissions"]:
        response = requests.get("{}/rest/submission/{}".format(vmray_url, vmray_submission), headers=vmray_headers)
        if response.status_code == 200:
            submission_finished_status = response.json()["data"].get("submission_finished", "")
            if not submission_finished_status:
                print(" Not all jobs for submission \033[91m{}\033[90m have finished. Please try again later.".format(vmray_submission))
                continue
            else:
                submission_finished = True
        else:
            print(" Unable to get submission \033[91m{}\033[90m status.".format(vmray_submission))
            print(response.text)

    # Jobs are finished, collect the results
    if submission_finished:
        print(" Submission \033[92m{}\033[90m is finished".format(sample["vmray"]["submissions"]))
        for vmray_sample in sample["vmray"]["samples"]:
            response = requests.get("{}/rest/sample/{}".format(vmray_url, vmray_sample), headers=vmray_headers)
            if response.status_code == 200:
                if response.json().get("result") == "ok":
                    response_json = response.json().get("data", {})
                    print(" Working on sample \033[92m{}\033[90m".format(vmray_sample))
                    samples[key]["vmray"]["sample_classifications"] = response_json.get("sample_classifications", [])
                    samples[key]["vmray"]["sample_highest_vti_score"] = response_json.get("sample_highest_vti_score", False)
                    samples[key]["vmray"]["sample_highest_vti_severity"] = response_json.get("sample_highest_vti_severity", False)
                    samples[key]["vmray"]["sample_severity"] = response_json.get("sample_severity", False)
                    samples[key]["vmray"]["sample_threat_names"] = response_json.get("sample_threat_names", [])
                    samples[key]["vmray"]["sample_verdict"] = response_json.get("sample_verdict", False)
                    samples[key]["vmray"]["sample_vti_score"] = response_json.get("sample_vti_score", False)

                    attribute = MISPAttribute()
                    attribute.value = response_json.get("sample_webif_url")
                    attribute.to_ids = False
                    attribute.category = "External analysis"
                    attribute.type = "link"
                    attribute.disable_correlation = True
                    attribute.comment = "Analysis with VMRay"
                    attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
                    if not "errors" in attribute_misp:
                        misp.add_object_reference(sample["fileobject"].add_reference(attribute_misp.uuid, "related-to"))
                        print(" Added link for {}".format(attribute.comment))
                        sample_vti_score = samples[key]["vmray"].get("sample_vti_score", 0)
                        if sample_vti_score:
                            tag = sandbox_calculate_risk_score(samples[key]["vmray"].get("sample_vti_score", 0))
                            if len(tag) > 0:
                                misp.tag(attribute_misp.uuid, tag)

                    report_md += "VMRay report: [{}]({})\n\n".format(response_json.get("sample_webif_url"), response_json.get("sample_webif_url"))
                    report_md += "- Verdict: **{}**\n".format(samples[key]["vmray"]["sample_verdict"])
                    report_md += "- Severity: **{}**\n".format(samples[key]["vmray"]["sample_severity"])
                    report_md += "- VTI score: **{}**\n".format(samples[key]["vmray"]["sample_vti_score"])
                    report_md += "- Highest severity: {}\n".format(samples[key]["vmray"]["sample_highest_vti_severity"])
                    report_md += "- Highest VTI score: {}\n".format(samples[key]["vmray"]["sample_highest_vti_score"])

                    if len(samples[key]["vmray"]["sample_classifications"]) > 0:
                        report_md += "#### Classifications\n"
                        for el in samples[key]["vmray"]["sample_classifications"]:
                            report_md += "- **{}**\n".format(el)

                    if len(samples[key]["vmray"]["sample_threat_names"]) > 0:
                        report_md += "#### Threat names\n"
                        for el in samples[key]["vmray"]["sample_threat_names"]:
                            report_md += "- **{}**\n".format(el)

                    #################################################################
                    # Now use the MISP VMRay import module
                    module_data = {
                        "module": "vmray_import",
                        "event_id": misp_event.id,
                        "config": {
                            "apikey": vmray_apikey,
                            "url": vmray_url,
                            "Sample ID": vmray_sample,
                            "Artifacts": "0",
                            "VTI": "1",
                            "IOCs": "1",
                            "Analysis Details": "0",
                        },
                        "data": ""
                    }
                    print(" Importing VMRay results")
                    module_comment = "From VMRay import for {}".format(key)
                    module_signatures = []
                    module_result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=module_data)
                    if "results" in module_result.json() and len(module_result.json()["results"]) > 0:
                        module_result_json = module_result.json()["results"]

                        for misp_attribute in module_result_json.get("Attribute", []):
                            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
                            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
                            if not "errors" in created_attribute:
                                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
                            else:
                                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
                        for misp_object in module_result_json.get("Object", []):
                            if misp_object["name"] in vmray_ignore_objects:
                                continue

                            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
                            if len(misp_object["Attribute"]) > 0:
                                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                                if not "errors" in created_object:
                                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                                    if vmray_signatures_in_report:
                                        if misp_object["name"] == "sb-signature":
                                            for attribute in misp_object.get("Attribute", []):
                                                if attribute.get("object_relation") == "signature":
                                                    module_signatures.append(attribute.get("value"))
                                else:
                                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))
                    if len(module_signatures) > 0:
                        report_md += "#### Signatures\n"
                        for el in module_signatures:
                            report_md += "- {}\n".format(el)

                    print(" Finished sample")

    samples[key]["vmray"]["report_md"] = report_md

Process VMRay 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
 Submission [13873809] is finished
 Working on sample 12102735
 Added link for Analysis with VMRay
 Importing VMRay results
 Got sandbox-report 
 Got sandbox-report 
 Got sandbox-report 
 Got sandbox-report 
 Got sandbox-report 
 Got sandbox-report 
 Got sb-signature 
 Got file 
 Got file 
 Got file 
 Got registry-key 
 Got file 
 Got file 
 Got file 
 Got file 
 Got file 
 Finished sample


## IN:2 Submit to Hybrid-Analysis

You can also submit the sample to [Hybrid-Analysis](https://www.hybrid-analysis.com/).

When you submit a sample to Hybrid-Analysis you have to specify the **environment** in which the sandbox needs to execute the sample. The comment section lists the available IDs for `environment_id`. Additionaly you can specify if you want to share sample with third parties (`no_share_third_party`), if you want to allow community access (`allow_community_access`) and if you want to use the hash lookup feature (`no_hash_lookup`).

In [9]:
# Available environments ID: 
#  400: 'Mac Catalina 64 bit (x86)', 
#  310: 'Linux (Ubuntu 20.04, 64 bit)', 
#  300: 'Linux (Ubuntu 16.04, 64 bit)', 
#  200: 'Android Static Analysis', 
#  160: 'Windows 10 64 bit', 
#  140: 'Windows 11 64 bit', 
#  120: 'Windows 7 64 bit', 
#  110: 'Windows 7 32 bit (HWP Support)', 
#  100: 'Windows 7 32 bit'

hybrid_environment_id = "120"

# When set to 'true', the sample is never shared with any third party. Default: true
hybrid_no_share_third_party = "true"

# When set to 'true', the sample will be available for the community. Default: true (Note: when 'no_share_third_party' is set to 'false', it won't be possible to set different value than 'true')
hybrid_allow_community_access = "false"

# Hash lookup
hybrid_no_hash_lookup = "false"

# Hybrid-analysis headers
hybridanalayis_headers = {
    "accept": "application/json",
    "api-key": hybridanalysis_apikey
}

### Submit

Execute the next cell to sumit the samples to Hybrid-Analysis. This will output the **Analysis** ID.

In [11]:
# Code block to submit to Hybrid-Analysis
if hybrid_environment_id in ["400", "310", "300", "200", "160", "140", "120", "110", "100"]:
    for key, sample in samples.items():
        print("Submit \033[92m{}\033[90m".format(key))

        tags = ""
        for tag in submission_tags:
            tags = "{} #{}".format(tags, tag)
        submit_parameters = {
            "environment_id": hybrid_environment_id,
            "comment": tags,
            "allow_community_access": hybrid_allow_community_access,
            "no_share_third_party": hybrid_no_share_third_party,
            "no_hash_lookup": hybrid_no_hash_lookup
        }
        submit_file = {"file": (key, io.BytesIO(sample["submitdata"]))}
        url = "https://www.hybrid-analysis.com/api/v2/submit/file"
        response = requests.post(url, headers=hybridanalayis_headers, files=submit_file, data=submit_parameters)

        if response.status_code == 201:
            analysis_id = response.json()["job_id"]
            samples[key]["hybrid-analysis"]["job_id"] = analysis_id
            print("Successfully submitted malware sample. Analysis ID: \033[92m{}\033[90m".format(analysis_id))
        else:
            print("Failed to submit malware sample. Status Code: \033[91m{}\033[90m".format(response.status_code))
            print(response.text)
else:
    print("Failed to submit malware sample. Not a valid environment_id: \033[91m{}\033[90m".format(environment_id))

Submit 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
Successfully submitted malware sample. Analysis ID: 654e2ff21b36c7c9e2016289


### Collect results

Execute the next cell to collect the results. The playbook will display the analysis state and only collect the results if all jobs are finished.

In [14]:
# Collect results from hybrid-analysis
for key, sample in samples.items():
    if samples[key]["hybrid-analysis"].get("job_id", False):
        state = False
        job_id = samples[key]["hybrid-analysis"]["job_id"]
        url = "https://www.hybrid-analysis.com/api/v2/"
        response = requests.get("{}report/{}/summary".format(url, job_id), headers=hybridanalayis_headers)
        samples[key]["hybrid-analysis"]["state"] = "Unknown"
        if response.status_code == 200:
            state = response.json().get("state", False)
            samples[key]["hybrid-analysis"]["state"] = state
        if state:
            print("Status for {} ({}): \033[92m{}\033[90m".format(key, job_id, samples[key]["hybrid-analysis"]["state"]))

        if samples[key]["hybrid-analysis"]["state"] == "SUCCESS":
            print("Process \033[92m{}\033[90m".format(key))

            response_json = response.json()

            samples[key]["hybrid-analysis"]["classification_tags"] = response_json.get("classification_tags", "")
            samples[key]["hybrid-analysis"]["type"] = response_json.get("type", "")
            samples[key]["hybrid-analysis"]["threat_score"] = response_json.get("threat_score", 0)
            samples[key]["hybrid-analysis"]["threat_level"] = response_json.get("threat_level", 0)
            samples[key]["hybrid-analysis"]["verdict"] = response_json.get("verdict", "")

            attribute = MISPAttribute()
            hybrid_link = "https://www.hybrid-analysis.com/sample/{}/{}".format(response_json.get("sha256"), job_id)
            attribute.value = hybrid_link
            attribute.to_ids = False
            attribute.category = "External analysis"
            attribute.type = "link"
            attribute.disable_correlation = True
            attribute.comment = "Analysis with Hybrid-analysis"
            attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
            if not "errors" in attribute_misp:
                misp.add_object_reference(sample["fileobject"].add_reference(attribute_misp.uuid, "related-to"))
                print(" Added link for {}".format(attribute.comment))

                threat_score = samples[key]["hybrid-analysis"].get("threat_score", 0)
                if threat_score is None:
                    threat_score = 0
                tag = sandbox_calculate_risk_score(threat_score)
                if len(tag) > 0:
                    misp.tag(attribute_misp.uuid, tag)

            attck_id = []
            for mitre_attcks in response_json.get("mitre_attcks", []):
                attck_id_descr = "{} : {} - {}".format(mitre_attcks.get("attck_id", ""), mitre_attcks.get("tactic", ""), mitre_attcks.get("technique", ""))
                if attck_id_descr not in attck_id:
                    attck_id.append(attck_id_descr)
                    tag = "misp-galaxy:mitre-attack-pattern=\"{} - {}\"".format(mitre_attcks.get("technique", ""), mitre_attcks.get("attck_id", ""))
                    misp.tag(misp_event.uuid, tag)
                    print(" Tag event with {}".format(tag))
            samples[key]["hybrid-analysis"]["mitre_attcks"] = attck_id

            signatures = []
            for signature in response_json.get("signatures", []):
                if not (signature.get("category", False) == "General"):
                    signature_descr = "{} - {} (ATT&CK: {})".format(signature.get("category", ""), signature.get("name", ""), signature.get("attck_id", ""))
                    if signature_descr not in signatures:
                        signatures.append(signature_descr)
            samples[key]["hybrid-analysis"]["signatures"] = signatures

            report_md = ""
            report_md += "Hybrid-Analysis report: [{}]({})\n\n".format(hybrid_link, hybrid_link)
            report_md += "- Verdict: **{}**\n".format(samples[key]["hybrid-analysis"]["verdict"])
            report_md += "- Threat score: **{}**\n".format(samples[key]["hybrid-analysis"]["threat_score"])
            report_md += "- Threat level: **{}**\n".format(samples[key]["hybrid-analysis"]["threat_score"])
            report_md += "- Type: **{}**\n".format(samples[key]["hybrid-analysis"]["type"])
            if len(samples[key]["hybrid-analysis"]["classification_tags"]) > 0:
                report_md += "#### Tags\n"
                for el in samples[key]["hybrid-analysis"]["classification_tags"]:
                    report_md += "- **{}**\n".format(el)

            if len(samples[key]["hybrid-analysis"]["mitre_attcks"]) > 0:
                report_md += "#### MITRE ATT&CK\n"
                for el in samples[key]["hybrid-analysis"]["mitre_attcks"]:
                    report_md += "- {}\n".format(el)
            if len(samples[key]["hybrid-analysis"]["signatures"]) > 0:
                report_md += "#### Signatures\n"
                for el in samples[key]["hybrid-analysis"]["signatures"]:
                    report_md += "- {}\n".format(el)

            print(" Finished sample")
            samples[key]["hybrid-analysis"]["report_md"] = report_md
        else:
            print(" Not all jobs for submission \033[91m{}\033[90m have finished. Please try again later.".format(job_id))

Status for 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run (654e2ff21b36c7c9e2016289): SUCCESS
Process 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
 Added link for Analysis with Hybrid-analysis
 Tag event with misp-galaxy:mitre-attack-pattern="Native API - T1106"
 Tag event with misp-galaxy:mitre-attack-pattern="Scheduled Task/Job - T1053"
 Tag event with misp-galaxy:mitre-attack-pattern="Windows Command Shell - T1059.003"
 Tag event with misp-galaxy:mitre-attack-pattern="Shared Modules - T1129"
 Tag event with misp-galaxy:mitre-attack-pattern="Malicious File - T1204.002"
 Tag event with misp-galaxy:mitre-attack-pattern="Scheduled Task - T1053.005"
 Tag event with misp-galaxy:mitre-attack-pattern="Inter-Process Communication - T1559"
 Tag event with misp-galaxy:mitre-attack-pattern="Component Object Model - T1559.001"
 Tag event with misp-galaxy:mitre-attack-pattern="Component Object Model Hijacking - T1546.015"
 Tag event with

## IN:3 Submit to VirusTotal

You can also submit the sample to [VirusTotal](https://www.virustotal.com). There are no configuration options for this sandbox.

### Submit

Execute the next cell to sumit the samples to VirusTotal. This will output the **Analysis ID**.

In [12]:
vt_headers = {
    "accept": "application/json",
    "x-apikey": virustotal_apikey
}
vt_url = "https://www.virustotal.com/api/v3/files"

for key, sample in samples.items():
    print("Submit \033[92m{}\033[90m".format(key))
    submit_file = {"file": (key, io.BytesIO(sample["submitdata"]))}
    response = requests.post(vt_url, headers=vt_headers, files=submit_file, data=submit_parameters)
    if response.status_code == 200:
        analysis_id = response.json()["data"]["id"]
        samples[key]["virustotal"]["analysis_id"] = analysis_id
        print("Successfully submitted malware sample. Analysis ID: \033[92m{}\033[90m".format(analysis_id))
    else:
        print("Failed to submit malware sample. Status Code: \033[91m{}\033[90m".format(response.status_code))
        print(response.text)

Submit 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
Successfully submitted malware sample. Analysis ID: N2M5ZGEyYTJjMjljMDQxMWU2MTQzMjJhMDJiYzQzZmE6MTY5OTcwMDcwNw==


### Collect results

Execute the next cell to collect the results. The playbook will display the analysis state and only collect the results if all jobs are finished.

In [15]:
# Collect results from VirusTotal
for key, sample in samples.items():
    if samples[key]["virustotal"].get("analysis_id", False):
        analysis_id = samples[key]["virustotal"]["analysis_id"]
        vt_url = "https://www.virustotal.com/api/v3/analyses/"
        result = requests.get("{}{}".format(vt_url, analysis_id), headers=vt_headers)
        vt_status = False
        if result.status_code == 200:
            if "data" in result.json() and len(result.json()["data"]) > 0:
                result_json = result.json()["data"]["attributes"]
                status = result_json.get("status", [])
                if status == "completed":
                    vt_status = True
        if vt_status:
            print("Process \033[92m{}\033[90m".format(key))
            report_md = ""
            vt_link = "https://www.virustotal.com/gui/file/{}".format(sample["sha256"])
            report_md += "VirusTotal report: [{}]({})\n\n".format(vt_link, vt_link)
            stats = result_json.get("stats", False)
            if stats:
                samples[key]["virustotal"]["harmless"] = stats.get("harmless", 0)
                samples[key]["virustotal"]["type-unsupported"] = stats.get("type-unsupported", 0)
                samples[key]["virustotal"]["suspicious"] = stats.get("suspicious", 0)
                samples[key]["virustotal"]["confirmed-timeout"] = stats.get("confirmed-timeout", 0)
                samples[key]["virustotal"]["timeout"] = stats.get("timeout", 0)
                samples[key]["virustotal"]["failure"] = stats.get("failure", 0)
                samples[key]["virustotal"]["malicious"] = stats.get("malicious", 0)
                samples[key]["virustotal"]["undetected"] = stats.get("undetected", 0)
                report_md += "#### Statistics\n"
                report_md += "- Malicious : **{}**\n".format(samples[key]["virustotal"]["malicious"])
                report_md += "- Suspicious : **{}**\n".format(samples[key]["virustotal"]["suspicious"])
                report_md += "- Harmless : **{}**\n".format(samples[key]["virustotal"]["harmless"])

            av_results = result_json.get("results", False)
            if av_results:
                av_results_detail = []
                for av_result in av_results:
                    if av_results[av_result].get("result", "") not in av_results_detail:
                        av_results_detail.append(av_results[av_result].get("result"))
                samples[key]["virustotal"]["results"] = av_results_detail
                if len(samples[key]["virustotal"]["results"]) > 0:
                    report_md += "#### AV results\n"
                    for el in samples[key]["virustotal"]["results"]:
                        report_md += "- {}\n".format(el)
     
            attribute = MISPAttribute()
            attribute.value = vt_link
            attribute.to_ids = False
            attribute.category = "External analysis"
            attribute.type = "link"
            attribute.disable_correlation = True
            attribute.comment = "Analysis with VirusTotal"
            attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
            if not "errors" in attribute_misp:
                misp.add_object_reference(sample["fileobject"].add_reference(attribute_misp.uuid, "related-to"))
                print(" Added link for {}".format(attribute.comment))
    
            print(" Finished sample")

            samples[key]["virustotal"]["report_md"] = report_md
        else:
            print(" Not all jobs for analysis \033[91m{}\033[90m have finished. Please try again later.".format(analysis_id))
    else:
        print(" Error while getting analysis \033[91m{}\033[90m.".format(analysis_id))

Process 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
 Added link for Analysis with VirusTotal
 Finished sample


## IN:4 Create MISP report for investigation

Create a report with the results of the dynamic malware analysis.

In [16]:
summary_iv = "## Investigation report\n\n"
current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")

at_least_one_sandbox = False
for key, sample in samples.items():
    summary_iv += "## Analysis for {}\n".format(key)
    summary_iv += "- Date: **{}**\n".format(formatted_date)
    summary_iv += "- MISP event: **{}** ({})\n".format(misp_event.info, misp_event.id)
    summary_iv += "### File details\n"
    summary_iv += " - MD5: {}\n".format(sample["md5"])
    summary_iv += " - SHA256: {}\n".format(sample["sha256"])

    if len(sample["vmray"].get("report_md", "")) > 0:
        summary_iv += "### VMRay\n"
        summary_iv += sample["vmray"]["report_md"]
        at_least_one_sandbox = True
    else:
        print("There are no \033[91mVMRay results\033[90m.")

    summary_iv += "\n\n"

    if len(sample["hybrid-analysis"].get("report_md", "")) > 0:
        summary_iv += "### Hybrid-Analysis\n"
        summary_iv += sample["hybrid-analysis"]["report_md"]
        at_least_one_sandbox = True
    else:
        print("There are no \033[91mHybrid-Analysis results\033[90m.")

    summary_iv += "\n\n"

    if len(sample["virustotal"].get("report_md", "")) > 0:
        summary_iv += "### VirusTotal\n"
        summary_iv += sample["virustotal"]["report_md"]
        at_least_one_sandbox = True
    else:
        print("There are no \033[91mVirusTotal results\033[90m.")

    summary_iv += "\n\n"

event_title = "Dynamic malware analysis - sandboxes"
if at_least_one_sandbox:
    print("Creating MISP report \033[92m{}\033[90m".format(event_title))
    chunk_size = 61500
    for i in range(0, len(summary_iv), chunk_size):
        chunk = summary_iv[i:i + chunk_size]
        event_report = MISPEventReport()
        event_title_edit = event_title
        if i > 0:
            event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
        event_report.name = event_title_edit
        event_report.content = chunk
        result = misp.add_event_report(misp_event.id, event_report)
        if "EventReport" in result:
            print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
        else:
            print("Failed to create report for \033[91m{}\033[90m.".format(event_title))
else:
    print("Failed to create report for \033[91m{}\033[90m.".format(event_title))

Creating MISP report Dynamic malware analysis - sandboxes
 Report ID: 799


# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

## EN:1 MISP indicators

The next section first **queries MISP for the indicators added to the MISP event** that is linked to the execution of this playbook.

The indicators are stored in the variable `indicator_table` (table format) and `indicator_raw_list` (in raw format) which is used in a later section to create the playbook summary.

In order to avoid too generic indicators, such as files temporarily created by processes, you can ignore all filename indicators that contain **.tmp** with the setting `ignore_filename_with_tmp`.

In [17]:
# Ignore filenames with ".tmp"
ignore_filename_with_tmp = True

# Get all the indicators for our event and store this is in a table. We can also use this for the summary.
print("Searching for indicators ...")
indicator_search = misp.search("attributes", uuid=misp_event.uuid, to_ids=True, pythonify=True)
indicator_raw_list = []
indicator_table = PrettyTable()
if len(indicator_search) > 0:
    indicator_table.field_names = ["Type", "Category", "Indicator", "Comment"]
    indicator_table.align["Type"] = "l"
    indicator_table.align["Category"] = "l"
    indicator_table.align["Indicator"] = "l"
    indicator_table.align["Comment"] = "l"
    indicator_table.border = True
    if ignore_filename_with_tmp:
        print("Ignoring filenames that contain .tmp")
    for indicator in indicator_search:
        if indicator.value not in indicator_raw_list:
            if ignore_filename_with_tmp and indicator.type == "filename" and ".tmp" in indicator.value:
                continue
            else:
                comment = indicator.comment
                if hasattr(indicator, 'object_relation'):
                    object_ind = misp.get_object(indicator.object_id, pythonify=True)
                    comment = "From '{}' object {} {}".format(object_ind.name, object_ind.comment.lower(), comment.lower())
                indicator_table.add_row([indicator.type, indicator.category, indicator.value, comment])
                indicator_raw_list.append(indicator.value)
    print("Got \033[92m{}\033[90m indicator(s) from the event \033[92m{}\033[90m ({}).\n".format(len(indicator_raw_list), misp_event.info, misp_event.id))
else:
    print("\033[93mNo indicators found in the event \033[92m{}\033[90m ({})".format(misp_event.info, misp_event.id))

Searching for indicators ...
Ignoring filenames that contain .tmp
Got 62 indicator(s) from the event Malware triage for incident in L2/L3 (3138).



### Raw list of MISP indicators

The indicators are now stored in `indicator_search` (as Python objects) and `indicator_raw_list` (in raw format, only the indicators). Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [18]:
if len(indicator_raw_list) > 0:
    print(indicator_table.get_string(sortby="Type"))
    print("\n\nIndicator list in raw format:")
    print("---------------------------------------------------")
    for el in indicator_raw_list:
        print("{}".format(el))
    print("---------------------------------------------------")

+----------------+-----------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+
| Type           | Category              | Indicator                                                                                                                        | Comment                                                                                                                                                |
+----------------+-----------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+
| authentihash   | 

## EN:2 Print the dynamic malware analysis report

Print the MISP report with the **dynamic** malware analysis results.

In [19]:
print(summary_iv)
# Or print with parsed markdown
# display_markdown(summary_iv, raw=True)

## Investigation report

## Analysis for 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4.do-not-run
- Date: **2023-11-11**
- MISP event: **Malware triage for incident in L2/L3** (3138)
### File details
 - MD5: 7c9da2a2c29c0411e614322a02bc43fa
 - SHA256: 8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4
### VMRay
VMRay report: [https://cloud.vmray.com/samples/12102735](https://cloud.vmray.com/samples/12102735)

- Verdict: **malicious**
- Severity: **malicious**
- VTI score: **96**
- Highest severity: malicious
- Highest VTI score: 96
#### Threat names
- **Mal/Generic-S**


### Hybrid-Analysis
Hybrid-Analysis report: [https://www.hybrid-analysis.com/sample/8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4/654e2ff21b36c7c9e2016289](https://www.hybrid-analysis.com/sample/8eff35b85b7f9a95534f58fd38ddb6a4d7d9b799fb8b1ea650f898b0ecf702d4/654e2ff21b36c7c9e2016289)

- Verdict: **malicious**
- Threat score: **100**
- Threat level: **100**
- Type: **

## EN:3 Create the summary of the playbook

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables are later used when sending information to Mattermost or TheHive.

In [20]:
summary = "# MISP Playbook summary\nDynamic malware analysis with MISP event: **{}** ({}/events/view/{}). ".format(misp_event.info, misp_url, misp_event.id)

summary += "\n"
summary += summary_iv
summary += "\n"
intro = summary

summary += "## Indicators\n\n"
summary += "### Indicators table\n\n"
if len(indicator_raw_list) > 0:
    indicator_table.set_style(MARKDOWN)
    summary += indicator_table.get_string(sortby="Type")
    summary += "\n\n\n" 
    summary += "### Indicators in **raw format**\n\n"
    for indicator in indicator_raw_list:
        summary += "{}\n\n".format(indicator)
    summary += "\n" 
else:
    summary += "There are no indicators"
summary += "\n\n"

summary += "\n\n"

print("The \033[92msummary\033[90m of the playbook is available.\n")

The summary of the playbook is available.



## EN:4 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [21]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [23]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

Summary is sent to Mattermost.



## EN:5 End of the playbook 

In [24]:
print("\033[92m End of the playbook")


 End of the playbook


## External references

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [VMRay](https://www.vmray.com/)
- [Hybrid-Analysis](https://www.hybrid-analysis.com/)
- [VirusTotal](https://www.virustotal.com)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
pefile
mwdblib
```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```